In [1]:
%pylab inline
%precision 6

Populating the interactive namespace from numpy and matplotlib


'%.6f'

In [2]:
import pandas as pd
import sklearn as skl
import sklearn

In [3]:
#you may download these functions from https://github.com/Apogentus/common
from common.serialization import pickle_load, pickle_save
from common.classes.Struct import Struct
from common.feature_transformations import get_one_hot_encoding
from common.functions import all_nums

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn import metrics

In [5]:
pd.options.display.max_colwidth=100
np.set_printoptions(linewidth=140,edgeitems=10)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
rcParams['figure.figsize'] = (8.0, 5.0)

In [6]:
Z_train = pd.read_csv('Data/train.csv', skipinitialspace=True, sep=',', encoding='latin', index_col='PostId')

In [7]:
Z_test = pd.read_csv('Data/test.csv', skipinitialspace=True, sep=',', encoding='latin', index_col='PostId')

In [8]:
train_inds = Z_train.index.values
test_inds = Z_test.index.values

In [9]:
len(train_inds)

63462

In [10]:
len(test_inds)

39678

In [10]:
Z=pd.concat([Z_train,Z_test])

In [11]:
del Z_train, Z_test

In [12]:
len(Z), len(train_inds)+len(test_inds)

(103140, 103140)

In [13]:
len(set(train_inds) & set(test_inds)), len(set(train_inds) | set(test_inds))

(0, 103140)

In [14]:
Z.rename(columns={'OpenStatus':'y'}, inplace=True)

In [15]:
X_train = Z.BodyMarkdown[train_inds].values
Y_train = Z.y[train_inds].values 
Y_train=Y_train.astype(str)

X_test = Z.BodyMarkdown[test_inds].values

In [16]:
unique(Y_train, return_counts=1)

(array(['not a real question', 'not constructive', 'off topic', 'open', 'too localized'], 
       dtype='<U19'), array([16459, 10421, 10634, 23017,  2931], dtype=int64))

In [17]:
len(X_train)

63462

In [18]:
len(Y_train)

63462

In [19]:
from sklearn.linear_model import SGDClassifier
clf = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='log', alpha=0.00001, max_iter=50)),
])

In [20]:
clf.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False))])

In [21]:
Y_hat = clf.predict(X_test)

In [22]:
unique(Y_hat, return_counts=1)

(array(['not a real question', 'not constructive', 'off topic', 'open', 'too localized'], 
       dtype='<U19'), array([10976,  3593,  5614, 19413,    82], dtype=int64))

# Submission

In [23]:
S = pd.DataFrame(Y_hat, columns=['OpenStatus'], index = Z.loc[test_inds].index)

In [24]:
S.head()

,OpenStatus
PostId,
2,open
4,open
8,not a real question
10,open
11,not a real question


In [25]:
S.to_csv('Data/baseline_solution.csv')  # accuracy>0.49 on test

Hints for further improvement:
 * tune parameters of baseline model
 * use information about title
 * use information about tags
 * use information about user reputation, duration on StackOverflow
 * generate your own features
 * did moderation policy change in 2012? adapt model scores to anticipate change in policy.